**텍스트를 문자로 변환**

In [ ]:
thor_review = "the action scenes were top notch in this movie. Thor has never been this epic in the MCU. He does some pretty epic sh*t in this movie and he is definitely not under-powered anymore. Thor in unleashed in this, I love that."

print(list(thor_review))

**텍스트를 단어로 변환**

In [ ]:
print(thor_review.split())

**N-그램 표현**

nltk패키지를 사용해서 2개 혹은 3개의 단어를 함께 묶어서 다룰 수 있다.

In [ ]:
from nltk import ngrams

print(list(ngrams(thor_review.split(),2)))

ngrams 함수는 첫번째 전달 인자로 단어 리스트, 두번째 전달 인자로 그룹화할 단어 수를 받는다.

In [ ]:
print(list(ngrams(thor_review.split(),3)))

N-그램의 문제점은 텍스트의 순서 정보를 상실한다는 점이다.  보통 얕은 머신 러닝 모델에서 주로 사용된다. RNN이나 Conv1D에서는 사용되지 않는다.

### 벡터화

**원-핫 인코딩**
각 토큰은 길이가 N인 벡터로 표현된다. 여기서 N은 어휘의 크기이다. 

In [ ]:
import numpy as np

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        self.length = 0
    def add_word(self, word):
        if word not in self.idx2word:
            self.idx2word.append(word)
            self.word2idx[word] = self.length + 1
            self.length += 1
        return self.word2idx[word]
    
    def __len__(self):
        return len(self.idx2word)
    
    def onehot_encoded(self, word):
        vec = np.zeros(self.length)
        vec[self.word2idx[word]] = 1
        return vec

> **__init__**에서는 word2idx 딕셔너리 객체를 만든다. 이 객체는 고유 단어를 idx와 함께 저장한다. **Idx2word** 리스트 객체는 고유 단어를 저장하고 **length** 변수는 문서에서 고유 단어의 총 개수를 저장한다.

> **add_word** 함수는 단어를 인수로 입력받는다. 입력된 단어가 기존에 idx2word에 없는 새로운 단어라면 추가한다.

> **onehot_encoded** 함수는 단어를 전달 인자로 바당 길이가 N인 벡터를 반환한다. 예를 들어 한 단어가 word2idx에서 idx가 2라면 길이가 N이고 엔덱스가 2인 요소의 값이 1이며 나머지는 인 0 0 1 0 0 0 0 0 0 0 와 같은 벡터가 반환된다.

In [ ]:
dic = Dictionary()

for tok in thor_review.split():
    dic.add_word(tok)

print(dic.word2idx)

In [ ]:
dic.onehot_encoded('were')

원-핫 인코딩 표현의 문제점 중 하나는 데이터에 대부분이 0 값이고, 어휘의 고유 단어수가 증가함에 따라 벡터의 크기가 급격하게 커진다는 것이다. 그래서 별로 쓰이지 않는다.

**워드 임베딩**

워드 임베딩이 가장 많이 사용되는 방식이다. 워드 임베딩은 부동 소수점 형태의 수로 채워진 밀집 벡터 형태를 갖는다. 의미가 유사한 단어는 비슷한 벡터를 갖도록 조정된다. 

## **감성 분류기로 워드 임베딩 학습시키기**

### IMDB 다운로드와 텍스트 토큰화

In [ ]:
pip install torchtext

In [ ]:
from torchtext import data
TEXT = data.Field(lower=True, batch_first=True, fix_length=40)
LABEL = data.Field(sequential=True)

TEXT는 모두 소문자로, 텍스트 토큰화를 수행하며, 공백을 없애고, 최대 길이를 20으로 제한했다. Field 생성자는 tokenize라는 인자를 갖는다. tokenize 인자의 기본값은 str.split이다. 

In [ ]:
from torchtext import datasets

train, test = datasets.IMDB.splits(TEXT, LABEL)

위 코드는 datasets의 IMDB 클래스는 데이터셋을 다운로드하고, 토큰화를 수행한 후, 데이터 셋을 학습과 테스트로 분할하는데 필요한 작업을 추상화한다. train.fields는 딕셔너리 객체를 포함한다. 이 딕셔너리 객체에서 TEXT가  키고, LABEL이 값이다.

In [ ]:
print('train.fields', train.fields)

In [ ]:
print(vars(train[0]))

### **어휘구축**

데이터가 로드되면 build_vocab을 호출하고 데이터의 어휘 생성에 필요한 인수를 전달할 수 있다.

In [ ]:
pip install glove-python

In [ ]:
from torchtext.vocab import GloVe 
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300),
                  max_size = 10000, min_freq = 10)
LABEL.build_vocab(train)

어휘 객체를 생성하는 부분에 train 객체를 전달하고, 사전에 학습된 임베딩을 이요해서 벡터를 초기화하도록 했다. build_vocab 메서드는 사전에 학습된 임베딩 데이터를 다운로드한다. 사전에 학습된 가중치를 이용하면 더 효과적이다.

max_size 는 어휘 객체의 크기를 제한하고 min_freq은 어휘에 추가될 단어의 최소 출현 빈도를 설정한다.

위코드는 출현 빈도가 10번 이상인 단어로 최대 크기가 1만개인 어휘 객체가 만들어진다.

In [ ]:
print(TEXT.vocab.freqs)

In [ ]:
print(TEXT.vocab.vectors)

In [ ]:
print(TEXT.vocab.stoi)
#단어와 단어의 인덱스를 관리하는 딕셔너리 객체에 접근

### 벡터 배치 생성

Torchtext는 모든 텍스트를 배치 처리하는 것을 지원하고, 단어를 인덱스 번호로 대체하는 BucketIterator를 제공한다. 

BucketIterater 인스턴스를 만드는 생성자는 batch_size, device(GPU, CPU 지정), shuffle(데이터를 섞을 것인지)

In [ ]:
train_iter, test_iter = data.BucketIterator.splits((train, test), 
                                                   batch_size=128,  shuffle=True)
# device = -1은 cpu, 기본값은 GPU이다.

In [ ]:
batch = next(iter(train_iter))
batch.text

In [ ]:
batch.label

### 임베딩으로 네트워크 모델 만들기

In [ ]:
import torch 
from torch import nn
from torch.nn import functional as F

class EmbNet(nn.Module):
    def __init__(self, emb_size, hidden_size1, hidden_size2 = 200):
        super().__init__()
        self.embedding = nn.Embedding(emb_size, hidden_size1) 
        #nn.Embedding 클래스 객체를 2개의 전달 인자로 초기화. 첫번째는 어휘의 크기, 두번째는 인자의 각 단어를 표현할 차원의 크기
        #프로그램을 빨리 실행하려면 임베딩크기가 작은 것이 좋지만, 시스템용으로 빌드할 때는 크기가 커야한다.
        self.fc = nn.Linear(hidden_size2, 3)
        #마지막에는 워드 임베딩을 3개 카테고리(양성, 음성, 판단 보류)에 대응시키는 선형 레이어가 사용된다.
    
    def forward(self, x): #입력 데이터가 처리되는 방법(예를 들어 배치 32 최대단어 20인 문장은 입력 데이터의 형상은 32*20이 된다. )
        embeds = self.embedding(x).view(x.size(0), -1) 
        #입력된 단어를 임베딩 벡터로 변환하는 룩업 테이블 역할.
        #워드 임베딩이 10차원으로 만들어진다면 데이터 형상이 32*20*10으로 나타나기 때문에 문장별로 평평하게 만들기 위해 view()를 사용한다. 
        #view에 전달되는첫번째 인자는 해당 크기를 그대로 유지 예를 계속 든다면 x.size(0), 즉 32를 유지하고 나머지는 합쳐 출력 데이터 형상이 32*200이 된다.
        out = self.fc(embeds)
        #DenseLayer는 워드 임베딩 레이어의 출력을 받아 3개의 카테고리에 대응을 시킨다.
        return F.log_softmax(out, dim=-1)

model = EmbNet(len(TEXT.vocab.stoi), 300, 12000)


### 모델 학습시키기

In [ ]:
def fit(epoch, model, data_loader, phase='training', volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx, batch in enumerate(data_loader):
        text, target = batch.text, batch.label
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.null_loss(output, target)

        running_loss += F.null_loss(output, target, size_average=False).data[0]
        preds = output.data.max(dim=1, keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == "target":
            loss.backward()
            optimizer.step()
    
    loss = running_loss / len(data_loader.dataset)
    accuracy = 100. * running_correct / len(data_loader.dataset)

    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss, accuracy

In [ ]:
model.embedding.weight.data = TEXT.vocab.vectors

In [ ]:
train_losses, train_accuracy = [], []
val_losses, val_accuracy = [], []

train_iter.repeat = False
test_iter.repeat = False


for epoch in range(1,10):
    epoch_loss, epoch_accuracy = fit(epoch, model, train_iter, phase="training")
    val_epoch_loss, val_epoch_accuracy = fit(epoch, model, test_iter, phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

## 사전 학습 워드 임베딩

### 임베딩 다운로드

In [ ]:
from torchtext.vocab import GloVe
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(train,)

### 모델에 임베딩 로딩하기
vectors 변수는 사전 학습된 임베딩을 포함하고 형상이(vocab_size, 차원)인 텐서를 반환한다.

임베딩 레이어의 가중치에 이 임베딩 가중치를 저장한다.

아래 코드를 통해 embeddings 레이어의 가중치에 사전 학습된 임베딩의 가중치를 할당할 수 있다.

In [ ]:
model.embedding.weight.data = TEXT.vocab.vectors

In [ ]:
class EmbNet(nn.Module):
    def __init__(self, emb_size, hidden_size1, hidden_size2 = 200):
        super().__init__()
        self.embedding = nn.Embedding(emb_size, hidden_size1)
        self.fc1 = nn.Linear(hidden_size2, 3)

    def forward(self, x):
        embeds = self.embedding(x).view(x.size(0),-1)
        out = self.fc1(embeds)
        return F.log_softmax(out, dim = -1)

model = EmbNet(len(TEXT.vocab.stoi),300,12000)

### 임베딩 레이어 가중치 고정

임베딩 가중치를 변경하지 못도록 할 때
> requires_grad 속성을 False로 설정해 이 가중치에 대한 기울기 변화를 추적할 필요가 없음을 알린다.

> 임베딩 레이어 파라미터가 옵티마이저로 전달되지 못하게 한다. 옵티마이저는 전달되는 모든 파라미터는 기울기를 관리하는 것으로 간주하기 때문...

임베딩 레이어의 가중치를 고정하고, 옵티마이저가 해당 매개변수를 사용하지 못하도록 하는 것이 포인트

In [ ]:
from torch import optim
model.embedding.weight.requires_grad = False
optimizer = optim.SGD([ param for param in model.parameters() if param.requires_grad ++ True], lr=0.001)
# requires_grad가 True일 경우에만 매개변수를 옵티마이저에 넘김.

## RNN

RNN은 레이블이 있는 순차 데이터에 이용하는 가장 강력한 모델.

대표적으로 분류, SwiftKey 키보드 등과같은 문장 생성 또는 하나의 시퀀스를 다른 시컨스로 변환하는 언어 번역에 활용 가능

RNN은 한 번에 텍스트 한 단어씩 보면서 사람과 비슷한 방식으로 작동한다. 

RNN은 독특한 레이어를 갖는 신경망 모델이다. 

기존에 신경망이 하나의 데이터를 한꺼번에 처리하는 방식이었다면, RNN은 하나의 데이터를 순차적이고 반복적으로 처리한다.

RNN은 순서대로 데이터를 처리하기 때문에 길이가 다른 벡터를 사용할 수 있다. 또한 길이가 다른 출력을 생성할 수도 있다. 

참고 ( http://karpathy.github.io/2015/05/21.rnn-effectiveness)


### RNN 작동 방식 이해

토르 영화 후기인 "the action scenes were top notch ..."를 입력한다면 첫 단어인 the를 가장 먼저 입력한다. RNN모델은 **상태 벡터**와 **출력 벡터**를 만든다.
**상태 벡터**는 영화 관람 후기의 다음 단어를 처리할 때 모델에 전달되고, 상태 벡터와 새로운 데이터가 결합해 새로운 상태 벡터와 출력 벡터가 만들어진다. 

코드로 이해를 돕는다면 아래와 같다.

    rnn = RNN(input_size, hidden_size, output_size)
    for i in range(len(thor_review)):
    output, hidden = rnn(thor_review[i], hidden)

hidden이 상태벡터를 나타내는 것이다. 

In [ ]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
    ## __init__에서 2개의 선형 레이어를 초기화한다. 출력과 상태 벡터 또는 히든 벡터를 계산한다.
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size) # 이게 아마 상태 계산 선형 레이어
        self.i2o = nn.Linear(input_size + hidden_size, output_size) # 이게 아마 출력 계산 선형 레이어
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        # 입력벡터와 히든 벡터를 cat함수로 결합해 출력 벡터와 숨겨진 상태를 생성하는 두 선형 레이어를 통과시킨다. 
        # 여기서 cat 함수는 텐서를 결합하며 두번째 인자 dim은 결합 방향을 설정.
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output) #출력 레이어는 log_softmax 함수를 적용
        return output, hidden
    
    def initHidden(self):
    # RNN을 처음 호출 할 경우 상태 벡터를 만드는 기능.
        return Variable(torch.zeros(1, self.hidden_size))


## LSTM 

RNN은 언어 번역, 텍스트 분류 등을 하는 알고리즘이다. 그러나 실무에서는 보통 RNN을 그대로 사용하진 않음.(기울기 소멸, 기울기 폭발 등의 문제가 존재)

그래서 RNN 변형 알고리즘인 LSTM 이나 GRU를 주로 사용한다.


### 장기 종속성
RNN은 이롡거으로 다음에 발생할 일에 대한 컨텍스트를 구축하기 위해 과거 데이터에서 필요한 모든 종속성을 학습해야 한다. 문장이 길어지면 컨텍스트 기억이 힘들다. 그래서 LSTM 내부에 다른 신경망을 추가해 이 문제를 해결한다. 


### LSTM
LSTM은 장기 의존성을 학습할 수 있다. LSTM에서 가장 중요한 것은 모든 반복에 걸쳐 전달되는 **셀 상태**이다.

가장 먼저 어떤 정보가 셀 상태에서 제거될지를 결정한다. 이 네트워크를 망각 게이트라고 한다. 이 네트워크는 활성 함수로 시그모이드를 사용한다. 셀 상태의 모든 요소는 0과 1 사이 값으로 출력된다.

다음 단계는 셀 상태에 어떤 정보를 추가할 지 결정한다. 여기서 입력게이트라 불리는 시그모이드 레이어는 업데이트 될 값을 결정한다. 셀 상태에 추가할 새 값을 만드는 tanh 레이어가 있다. 

다음 단계에서는 입력게이트와 tanh이 만든 값을 결합한다. 망각 게이트와 Ct의 같은 위치 요소끼리의 곱과 입력게이트와 tanh로 활성화된 Ct의 같은 요소끼리의 곱의 합으로 셀 상태를 업데이트 할 수 있다. 

마지막으로 출력을 결정한다. 출력은 필터링된 버전의 셀 상태이다. 

LSTM 이론 정보 참고
(http://colah.github.io/posts/2015-08-Understanding-LSTMs)
(http://brohrer.github.io/how_rnns_lstm_work.html)

### 네트워크로 감성 분류기 만들기 
1. 데이터 준비
2. 배치 처리기 만들기
3. 네트워크 생성
4. 모델 교육

**데이터 준비하기**


In [ ]:
from torchtext import data, datasets

TEXT = data.Field(lower = True, fix_length=200, batch_first=False)
LABEL = data.Field(sequential=False,)

train, test = datasets.IMDB.splits(TEXT, LABEL)

TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(train,)

**배치 처리기 생성하기**

배치 처리기 생성할 때 torchtext의BucketIterator를 사용한다.

In [ ]:
train_iter, test_iter = data.BucketIterator.splits((train,test), batch_size=32)
train_iter.repeat = False
test_iter.repeat =  False

**네트워크 생성하기**



In [ ]:
from torch import nn as nn
from torch.nn import functional as F

class IMDBRnn(nn.Module):

    def __init__(self, vocab, hidden_size, n_cat, bs=1, nl=2):
    
        super().__init__()
        self.hidden_size = hidden_size
        self.bs = bs
        self.nl = nl
        self.e = nn.Embedding(n_vocab, hidden_size) # 생성자인 __init__메서드에서 어휘의 크기와 hidden_size 크기의 임베딩 레이어를 만든다.
        self.rnn = nn.LSTM(hidden_size, hidden_sizem, nl) 
        self.fc2 = nn.Linear(hidden_size, n_cat)
        self.softmax = nn.LogSoftmax(dim=-1) #LogSoftmax는 선형 레이어의 결과를 확률로 변환한다.

    def forward(self, inp):
        bs = inp.size()[1]
        if bs != self.bs: # bs는 batch_size
            self.bs = bs
        e_out = self.e(inp) #임베디드 레이어 통과
        h0 = c0 = Variable(e_out.data.new(*(self.nl, self.bs, self.hidden_size)).zero())
        rnn_o,_ = self.rnn(e_out, (h0, c0))
        rnn_o = rnn_o[-1]
        fc = F.droput(self.fc2(rnn_o), p=0.8)
        return self.softmax(fc)

forward 메서드는 크기가 [200,32]의 입력 데이터를 임베디드 레이어에 통과시킨다. 
배치의 각 토큰은 임베딩으로 대체되고, 크기는 [200, 32, 100]으로 바뀐다. 여기서 100은 임베딩 차원이다. 

LSTM 렝이어는 2개의 hidden 변수와 임베디드 레이어의 출력을 입력받는다. 
여기서 두 hidden 변수는 임베딩 출력과 동일한 유형이어야하며, 크기는 [num_layers, batch_size, hidden_dim] 형상의 출력을 생성한다. LSTM은 시퀀스의 데이터를 처리하고 [Sequence_length, batch_size, hidden_size] 형상의 출력을 생성한다. 여기서 각 시퀀스 인덱스는 해당 시퀀스의 출력을 나타낸다. 이 때, 마지막 시퀀스의 출력을 가져온다. 이 시퀀스의 형상은 [batch_size, hideen_dim]이고, 이 출력을 선형 레이어에 전달해 출력 범주에 대응시킨다. 모델이 과대적합되는 경향이 있으므로 드롭아웃 레이어를 추가한다. 드롭아웃 확률은 모델을 학습시키는 과정에서 조정될 수 있다. 

**모델 학습시키기**

In [ ]:
model = IMDBRnn(n_vocab, n_hidden, 3, bs=32)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

def fit(epoch, model, data_loader, phase="training", voatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    
    running_loss = 0.0
    running_correct = 0
    for batch_idx, batch in enumerate(data_loader):
        text, target = batch.text, batch.label
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(ouput, target)

        running_loss += F.nll_loss(output, target, size_average=False).data[0]
        preds = output.data.max(dim=1, keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss / len(data_loader.dataset)
    accuracy = 100. * running_correct / len(data_loader.dataset)

    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct} / {len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss, accuracy

train_losses, train_accuracy = [], []
val_losses, val_accuracy = [], []

for epoch in range(1,5):
    epoch_loss, epoch_accuracy = fit(epoch, model, train_iter, phase='training')
    val_epoch_loss, val_epoch_accuracy = fit(epoch, model, test_iter, phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

## 시퀀스데이터와 CNN

CNN은 이미지의 피처를 학습하는 컴퓨터 비전에서도 쓰인다. 이미지의 경우 2차원을 컨볼루션하며 동작한다. 같은 방식으로 시간도 컨볼루션 피처에 넣을 수 있다. CNN을 통해 텍스트 분류를 구축해보자!

### 시퀀스 데이터를 위한 1차원 컨볼루션 이해

**네트워크 만들기**




In [ ]:
class IMDBCnn(nn.Module):
    def __init__(self, vocab, hidden_size, n_cat, bs=1, kernel_size=3, max_len=200):
        super().__init__()
        self.hidden_size = hidden_size
        self.bs = bs
        self.e = nn.Embedding(n_vocab, hidden_size)
        #LSTM레이어 대신 Conv1d 레이어와 AdqptiveAvgPool1d 레이어를 사용했다.
        self.cnn = nn.Conv1d(max_len, hidden_size, kernel_size) # 컨볼루션 레이어의 입력데이터는 시퀀스 길이, 출력 크기는 hidden_size, 그리고 커널 크기는 3으로 만들어졌다.
        self.avg = nn.AdaptiveAvgPool1d(10) # 컨볼루션 레이어의 출력 형상이 변경되면 선형 레이어의 차원을 변경해야하기 때문에 여러 크기의 입력을 받아 고정된 길이의 출력을 만드는 AdaptiveAvg 레이어를 사용한다. 
        self.fc = nn.Linear(1000, n_cat)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, inp):
        bs = inp.size()[0]
        if bs != self.bs:
            self.bs = bs
        e_out = self.e(inp)
        cnn_o = self.cnn(e_out)
        cnn_avg = self.avg(cnn_o)
        cnn_avg = cnn_avg.view(self.bs -1)
        fc = F.dropout(self.fc(cnn_avg), p=0.5)
        return self.softmax(fc)

**모델 학습시키기**

    train_losses, train_accuracy = [], []
    val_losses, val_accuracy = [], []

    train_iter.repeat = False
    test_iter.repeat = False


    for epoch in range(1,10):
        epoch_loss, epoch_accuracy = fit(epoch, model, train_iter, phase="training")
        val_epoch_loss, val_epoch_accuracy = fit(epoch, model, test_iter, phase='validation')
        train_losses.append(epoch_loss)
        train_accuracy.append(epoch_accuracy)
        val_losses.append(val_epoch_loss)
        val_accuracy.append(val_epoch_accuracy)